In [71]:
import os
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import openpyxl
import json
from pathlib import Path
base_path = Path(r'%notebook').absolute().parent.parent.parent
with open(fr"{base_path}\config.json", "r") as f:
    config = json.load(f)

In [72]:
url = 'https://www.books.com.tw/web/sys_saletopb/books'
req = requests.get(url)
soup = BeautifulSoup(req.text, "html.parser")
raw_data = soup.select('ul[class="clearfix"] li[class="item"]') + soup.select('ul[class="clearfix"] li[class="item last"]')

In [73]:
df_output = pd.DataFrame()
for idx, ele in enumerate(raw_data):

    rank = ele.select('strong[class="no"]')[0].text

    print(rank)

    book_name = ele.select('h4')[0].text
    author_data = ele.select('ul[class="msg"] li a')  
    author = author_data[0].text if len(author_data) > 0 else ''

    price_data = ele.select('li[class="price_a"] strong')
    
    if len(price_data) > 1:
        discount_rate = price_data[0].text ## 可能沒有
        price = price_data[1].text
    else:
        discount_rate = 100
        price = price_data[0].text

    df_ele = pd.DataFrame(data=[[rank, book_name, author, discount_rate, price]],
                          columns=['rank', 'book_name', 'author', 'discount_rate', 'price'])
    
    df_output = pd.concat([df_output, df_ele], ignore_index=True)


1
2
3
4
6
7
8
9
11
12
13
14
16
17
18
19
21
22
23
24
26
27
28
29
31
32
33
34
36
37
38
39
41
42
43
44
46
47
48
49
51
52
53
54
56
57
58
59
61
62
63
64
66
67
68
69
71
72
73
74
76
77
78
79
81
82
83
84
86
87
88
89
91
92
93
94
96
97
98
99
5
10
15
20
25
30
35
40
45
50
55
60
65
70
75
80
85
90
95
100


In [74]:
df_output['rank'] = df_output['rank'].astype(int)
df_output['discount_rate'] = df_output['discount_rate'].astype(int)
df_output['price'] = df_output['price'].astype(int)

df_output = df_output.sort_values(by='rank') 


## excel output
output_path = rf"book_seller_rank_{datetime.today().strftime('%Y%m%d')}.csv"
df_output.to_csv(output_path, index=False, encoding='utf-16', sep='\t')
